In [39]:
import cv2
import numpy as np
import tensorflow as tf
import torch
from tensorflow.keras.layers import TFSMLayer

# YOLO 
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
yolo_model.classes = [0] 

# Load the model
emotion_model_path = r"C:\Users\andre\Documents\Faculdade\Mestrado\1ano_1sem\VC\project\pila"
emotion_model = TFSMLayer(emotion_model_path, call_endpoint='serving_default')
emotion_labels = ['Angry', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

def recalibrate_probabilities(probabilities, temperature=1.0, epsilon=1e-9):
    
    probabilities = np.clip(probabilities, epsilon, 1 - epsilon)
    probabilities_log_normalized = probabilities / np.sum(probabilities)

    scaled_logits = np.log(probabilities_log_normalized + epsilon) / temperature
    
    recalibrated_probs = np.exp(scaled_logits) / np.sum(np.exp(scaled_logits))
    return recalibrated_probs

def moving_average(new_probs, prev_probs, alpha=0.7):
    """Combina probabilidades com uma média móvel."""
    return alpha * prev_probs + (1 - alpha) * new_probs

cap = cv2.VideoCapture(0)
prev_probs = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
        
    results = yolo_model(frame)
    detections = results.xyxy[0].cpu().numpy()

    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection
        if conf < 0.5:
            continue

        face = frame[int(y1):int(y2), int(x1):int(x2)]
        if face.size == 0:
            continue
            
        face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        face_resized = cv2.resize(face_gray, (224, 224)) / 255.0  # Normalizar

        face_normalized = (face_resized - np.min(face_resized)) / (np.max(face_resized) - np.min(face_resized) + 1e-9)
        face_input = np.expand_dims(np.expand_dims(face_normalized, axis=-1), axis=0).astype('float32')

        predictions = emotion_model(face_input)
        probabilities = next(iter(predictions.values())).numpy()

        recalibrated_probs = recalibrate_probabilities(np.squeeze(probabilities), temperature=3.0)

        if prev_probs is None:
            prev_probs = np.zeros_like(recalibrated_probs)

        smoothed_probs = moving_average(recalibrated_probs, prev_probs)
        prev_probs = smoothed_probs

        print(f"Original Probabilities: {probabilities}")
        print(f"Recalibrated Probabilities: {recalibrated_probs}")
        print(f"Smoothed Probabilities: {smoothed_probs}")

        emotion = emotion_labels[np.argmax(smoothed_probs)]

        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, emotion, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    cv2.imshow("Emotion Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to close
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\andre/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-12-10 Python-3.10.4 torch-2.1.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Original Probabilities: [[    0.33132     0.21892    0.040956     0.12423     0.24409    0.040472  1.2133e-05]]
Recalibrated Probabilities: [     0.2211     0.19258     0.11014     0.15944     0.19969     0.10971   0.0073426]
Smoothed Probabilities: [   0.066331    0.057773    0.033042    0.047831    0.059908    0.032912   0.0022028]
Original Probabilities: [[    0.33131     0.21874    0.040929     0.12446     0.24399    0.040567  1.2232e-05]]
Recalibrated Probabilities: [    0.22108     0.19251     0.11011     0.15952     0.19965     0.10978   0.0073619]
Smoothed Probabilities: [    0.11276    0.098193    0.056162    0.081338     0.10183    0.055972   0.0037505]
Original Probabilities: [[    0.33128     0.21872    0.040931     0.12449       0.244    0.040565  1.2236e-05]]
Recalibrated Probabilities: [    0.22107      0.1925     0.11011     0.15953     0.19965     0.10978   0.0073625]
Smoothed Probabilities: [    0.14525     0.12649    0.072346      0.1048     0.13118    0.072114   0.0

Original Probabilities: [[    0.33136     0.21857    0.040925     0.12454     0.24404     0.04056   1.225e-05]]
Recalibrated Probabilities: [    0.22109     0.19246      0.1101     0.15955     0.19966     0.10977   0.0073653]
Smoothed Probabilities: [    0.22106     0.19244     0.11009     0.15954     0.19966     0.10975   0.0073624]
Original Probabilities: [[    0.33129     0.21856    0.040915     0.12459     0.24406    0.040567  1.2261e-05]]
Recalibrated Probabilities: [    0.22107     0.19245     0.11009     0.15957     0.19966     0.10978   0.0073675]
Smoothed Probabilities: [    0.22106     0.19245     0.11009     0.15955     0.19966     0.10976   0.0073639]
Original Probabilities: [[    0.33124     0.21848    0.040893     0.12467     0.24415    0.040563  1.2272e-05]]
Recalibrated Probabilities: [    0.22106     0.19243     0.11007     0.15961     0.19969     0.10978   0.0073697]
Smoothed Probabilities: [    0.22106     0.19244     0.11008     0.15957     0.19967     0.10976   0.0